In [ ]:
# !pip install -q -r requirements.txt
from __reading import *

In [ ]:
match_sent=next(yield_sents())

In [ ]:
printm('\n'.join([f'* `{k}`: {v}`' for k,v in match_sent.items()]))

In [ ]:
def yield_sents_by_match(match_sent, match_keywords=MUST_CONTAIN, **kwargs):
    text_iter = yield_sents(must_contain=match_keywords, **kwargs)
    for text in text_iter:
        for keyword in match_keywords:
            if keyword in text['sent']:
                for match_d in process_match(text, keyword):
                    yield {
                        **text,
                        **match_d
                    }


def process_match(text_d: dict, keyword: str):
    sent_str = text_d['context1']
    assert keyword in sent_str, f'keyword {keyword} not in {sent_str}'
    
    tokens = sent_str.split()
    for token_id, token in enumerate(tokens):
        if not keyword.lower() in token.lower():
            continue
        
        # get prev alpha token
        nexttok = ""
        prevtok = ""
        for tok in reversed(tokens[:token_id]):
            if any(x.isalpha() for x in tok):
                prevtok = tok
                break
        for tok in tokens[token_id+1:]:
            if any(x.isalpha() for x in tok):
                nexttok = tok
                break
        
        yield {
            "token_num": token_id,
            'token0': strip_punct(prevtok),
            'token1': strip_punct(keyword),
            'token2': strip_punct(nexttok),
            "word": keyword,
            **text_d,
            **(WORD2DATA[keyword] if keyword in WORD2DATA else {})
        }



In [ ]:
iterr = yield_sents_by_match(match_sent)


In [ ]:
out = next(iterr)
printm('\n'.join([f'* `{k}`: {v}' for k,v in out.items()]))

In [ ]:
ld = list(iterr)

In [15]:
df = pd.DataFrame(ld)
df

,url,page_num,sent_num,sent,context0,context1,context2,token_num,token0,token1,...,fpm_coca_fic,fpm_coca_mag,fpm_coca_news,fpm_coca_spok,freq_COCA,lemma,null,perc_caps,pos,rank
0,http://www.jstor.org/stable/512209,1,10,"Some generalization may be in order, but soone...",A Reader's Guide to the Metaphysical Poets.\nB...,"Some generalization may be in order, but soone...","In his two opening chapters, Professor William...",16,his,reader,...,15.79,38.59,19.54,7.79,16779,reader,@,0.25,nn1,2548
1,http://www.jstor.org/stable/512209,1,19,Whereas in the sections on the short poems of ...,"In his examination of individual poems, howeve...",Whereas in the sections on the short poems of ...,"Even were the metaphrasing better done, howeve...",34,the,reader,...,15.79,38.59,19.54,7.79,16779,reader,@,0.25,nn1,2548
2,http://www.jstor.org/stable/512209,1,21,Turn some of Donne's poems into prose and they...,But Mr. Williamson is not faithful even to his...,Turn some of Donne's poems into prose and they...,He has a good deal to say about wit in his ope...,56,the,reader,...,15.79,38.59,19.54,7.79,16779,reader,@,0.25,nn1,2548
3,http://www.jstor.org/stable/437950,3,8,A great virtue of Beginnings is that it sharpe...,That this problem remains unresolved has to do...,A great virtue of Beginnings is that it sharpe...,One would also like to know how Said's concept...,10,the,reader,...,15.79,38.59,19.54,7.79,16779,reader,@,0.25,nn1,2548
4,http://www.jstor.org/stable/437950,3,10,To express this wish is not to fault Said's bo...,Said deserves every credit for avoiding such p...,To express this wish is not to fault Said's bo...,"It is, if you will, a kerygmatic work with a m...",27,the,reader,...,15.79,38.59,19.54,7.79,16779,reader,@,0.25,nn1,2548
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
207167,http://www.jstor.org/stable/3723444,2,9,"Crabbe is fond of pointing a moral, eagerly ur...",There is the description of Judge Bott in The ...,"Crabbe is fond of pointing a moral, eagerly ur...",Without doubt there is a certain general simil...,32,the,reader,...,15.79,38.59,19.54,7.79,16779,reader,@,0.25,nn1,2548
207168,http://www.jstor.org/stable/3723444,2,9,"Crabbe is fond of pointing a moral, eagerly ur...",There is the description of Judge Bott in The ...,"Crabbe is fond of pointing a moral, eagerly ur...",Without doubt there is a certain general simil...,40,the,reader,...,15.79,38.59,19.54,7.79,16779,reader,@,0.25,nn1,2548
207169,http://www.jstor.org/stable/3723444,3,4,The excessive sentimentality which makes much ...,L. H. C. THOMAS 353 which really represented a...,The excessive sentimentality which makes much ...,How different from the tone adopted by Droste ...,13,present-day,reader,...,15.79,38.59,19.54,7.79,16779,reader,@,0.25,nn1,2548
207170,http://www.jstor.org/stable/3723444,4,4,Here Droste invokes pity for an unfortunate wh...,354 'Die Judenbuche' and English Literature Wo...,Here Droste invokes pity for an unfortunate wh...,"She hopes that her reader, having perused the ...",16,the,reader,...,15.79,38.59,19.54,7.79,16779,reader,@,0.25,nn1,2548


In [18]:
df.to_pickle('data/corpus_instances.pkl.gz')